In [ ]:
import math, os

import matplotlib.pyplot as plt
import numpy as np

import amrex.space3d as amr
import impactx
from impactx import ImpactX, RefPart, distribution, elements

In [ ]:
# set before kernel startup
#os.environ["OMP_NUM_THREADS"] = "4"

In [ ]:
pp_amr = amr.ParmParse("amr")
pp_amr.add("max_level", 3)
pp_amr.addarr("n_cell", [32, 32, 32])
#pp_amr.addarr("n_cell", [64, 64, 64])
pp_amr.addarr("ref_ratio", [2, 2, 2])

In [ ]:
sim = ImpactX()

In [ ]:
sim.particle_shape = 2  # B-spline order
sim.space_charge = True
sim.dynamic_size = True
sim.prob_relative = [1.5, 1.1, 0.7, 0.5]
sim.diagnostics = False

In [ ]:
sim.domain

In [ ]:
#sim.domain = amr.RealBox((-15e-4, -15e-4, -3e-6), (15e-4,  15e-4,  3e-6))
#sim.domain

In [ ]:
sim.init_grids()

In [ ]:
# load a 2 GeV electron beam with an initial
# unnormalized rms emittance of 2 nm
energy_MeV = 250  # reference energy
bunch_charge_C = 1.0e-9  # used with space charge
npart = 10000  # number of macro particles

#   reference particle
ref = sim.particle_container().ref_particle()
ref.set_charge_qe(-1.0).set_mass_MeV(0.510998950).set_energy_MeV(energy_MeV)

#   particle bunch
distr = distribution.Gaussian(
    sigmaX=4.472135955e-4,
    sigmaY=4.472135955e-4,
    sigmaT=9.12241869e-7,
    sigmaPx=0.0,
    sigmaPy=0.0,
    sigmaPt=0.0,
)
sim.add_particles(bunch_charge_C, distr, npart)

In [ ]:
beam = sim.particle_container()
beam.min_and_max_positions()

In [ ]:
# design the accelerator lattice
sim.lattice.extend([
    elements.Drift(ds=6.0e-9, nslice=1),
    #elements.BeamMonitor('diag'),
])

In [ ]:
# run simulation
sim.evolve()

In [ ]:
pc = sim.particle_container()
pc.TotalNumberOfParticles()
# should be 10k if the box is not cut

In [ ]:
rho = sim.rho(lev=0)
rho.size

In [ ]:
#rho = sim.rho(lev=1)
#rho.size

In [ ]:
beam = sim.particle_container()
beam.min_and_max_positions()

In [ ]:
sim.resize_mesh()

In [ ]:
for lev in range(sim.finest_level + 1):
    print(lev)
    gm = sim.Geom(lev=lev)
    print(gm.ProbDomain())
    dm = gm.Domain()
    print(dm, dm.size[2])

In [ ]:
def plot_rho(lev):
    rho = sim.rho(lev=lev)

    gm = sim.Geom(lev=lev)
    dm = gm.Domain()
    dr = gm.data().CellSize()  # dz is in time now (s-based!)
    dV = np.prod(dr)
    
    rs = rho.sum_unique(comp=0, local=False)
    beam_charge = dV * rs  # in C
    print(f"dr={dr}, dV={dV}")
    print(f"beam_charge={beam_charge}")

    half_z = dm.size[2] // 2  # order: x,y,z
    # lvl 0 only:
    # half_z = sim.n_cell[2] // 2  # order: x,y,z

    ng = rho.nGrowVect
    print(f"ng={ng}")
    save_png = False
    for mfi in rho:
        f = plt.figure()
        ax = f.gca()
        bx = mfi.validbox()
        print(f"bx={bx}")
        rbx = amr.RealBox(bx, dr, gm.ProbLo())
        print(rbx)

        arr = rho.array(mfi)
        arr_np = np.array(arr, copy=False)  # indices: comp, z, y, x

        # shift box to zero-based local mfi index space
        half_z_local = half_z - bx.lo_vect[2]
        bx.shift(bx.lo_vect * -1)
        # check if the current tile contains the half-z plane
        if half_z_local < 0 or half_z_local > arr_np.shape[2]:
            continue

        comp = 0
        mu = 1.0e6  # m->mu
        im = ax.imshow(
            # arr_np[comp, half_z, ...] * dV,  # including guard
            arr_np[comp, half_z_local, ng[1] : -ng[1], ng[0] : -ng[0]],
            #* dV,  # w/o guard
            origin="lower",
            aspect="auto",
            extent=[rbx.lo(0) * mu, rbx.hi(0) * mu, rbx.lo(1) * mu, rbx.hi(1) * mu],
        )
        cb = f.colorbar(im)
        cb.set_label(r"charge density  [C/m$^3$]")
        ax.set_xlabel(r"$x$  [$\mu$m]")
        ax.set_ylabel(r"$y$  [$\mu$m]")
        #if save_png:
        #    plt.savefig("charge_deposition.png")
        #else:
        #    plt.show()

In [ ]:
plot_rho(lev=0)

In [ ]:
plot_rho(lev=1)

In [ ]:
plot_rho(lev=2)

In [ ]:
plot_rho(lev=3)

In [ ]:
%matplotlib widget

In [ ]:
from openpmd_viewer import OpenPMDTimeSeries

In [ ]:
ts = OpenPMDTimeSeries("diags/diag")